<h1 align="center">Bitcoin Encryption Standard: Electrum BIE1 ECIES</h1>
<h3 align="center">Benjamin Watts</h3>
<h3 align="center">Research Team (Intern)</h3>
<br>


<br>The encryption protocol we follow is Electrum BIE1 ECIES.<br>
<br>Setup:<br><br>
Alice has a public/private keypair $P_{A}^{DH}=S_{A}^{DH}*G$ and Bob has a keypair $P_{B}^{DH}=S_{B}^{DH}*G$ We assume that Alice and Bob have exchanged public keys off-chain and now Alice wishes to send an encrypted message to Bob on-chain, for example within an OP_RETURN statement. Alice will sign the on-chain transaction with a sperate keypair $P_{A}^{Tx} = S_{A}^{Tx}*G$ 

Summary table:
<h3 align="center">1. Establishing a Shared Secret</h3> 

|||
|:----------------------------------:|:------------------------------------------------------------:|
|Alice's ECDH Key                    | $P_{A}^{DH}$|
|Bob's ECDH Key                      | $P_{B}^{DH}$|
|ECDH Shared Secret Point            | $S_{AB} = P_{A}^{DH} ⋅ P_{B}^{DH} = P_{B}^{DH} ⋅ P_{A}^{DH}$|
|ECDH Shared Secret Data             | $S_{AB}$ **(compressed)**|
|Hash Function for Shared Secret Data| **SHA-512**|

<h3 align="center">2. Message Encryption and Authentication</h3> 

|||
|:----------------------------------:|:------------------------------------------------------------:|
|Encryption Algorithm                | **AES 128-BIT CBC mode**                                     |
|Initialization Vector (IV)          | **First 128-bits of** $H(S)$                                 |
|Encryption Key                      | **Second 128-bits of** $H(S)$                                |
|HMAC Key                            | **Final 256-bits of** $H(S)$                                 |
|Message Authentication              | **First 128-bits of HMAC-SHA256 using entire ciphertext**    |

<h3 align="center">3. Transaction Composition</h3> 

|||
|:----------------------------------:|:------------------------------------------------------------:|
|Tx Signature Key                    | $P_{A}^{Tx}$                                                 |
|Encryption Data Packet              | $P_{A}^{DH}$ **, ciphertext, HMAC**                          |
|Protocol Flag                       | **BIE1**                                                     |
|OP_RETURN Payload                   | **(BIE1)(**$P_{A}^{DH}$ **, ciphertext, HMAC)**              |

This protocol adheres to the following principles:<br>
1. Transaction signing keys are separate from encryption (Diffie-Hellman) keys. Advantages:
>a) Increased privacy and security.<br>
>b) The OP_RETURN payload can be used for secure off-chain messaging as well.<br>
>c) The protocol can easily be adapted to use Diffie-Hellman keys that do no belong to the bitcoin secp256k1 conventions. For example, with keys that have 256-bit security<br><br>
2. A new encryption key is used for each plaintext. Advantages:
>a) Assuming (1), if an audit takes place, then one can hand over decryption keys that will give the auditor power to view one plaintext only.



## Libraries and Modules

In [55]:
import binascii 
import hashlib
import ecdsa
import codecs
import hmac
import base64

from Crypto.Cipher import AES
from pkcs7 import PKCS7Encoder

## Elliptic Curve Field Setup

All key-related operations for Bitcoin occur within the finite group of secp256k1.<br>
Secp256k1, http://www.oid-info.com/get/1.3.132.0.10<br>

In [64]:
p_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
r_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
b_ec = 0x0000000000000000000000000000000000000000000000000000000000000007
a_ec = 0x0000000000000000000000000000000000000000000000000000000000000000
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
curve_secp256k1 = ecdsa.ellipticcurve.CurveFp(p_ec, a_ec, b_ec)
generator_secp256k1 = ecdsa.ellipticcurve.Point(curve_secp256k1, Gx, Gy, r_ec)
oid_secp256k1 = (1, 3, 132, 0, 10)
SECP256k1 = ecdsa.curves.Curve("SECP256k1", curve_secp256k1, generator_secp256k1, oid_secp256k1)

## Functions

In [57]:
def get_point_pubkey(point_input):
    if (point_input.y() % 2) == 1:
        result = '03' + '%064x' % point_input.x()
    else:
        result = '02' + '%064x' % point_input.x()
    return binascii.hexlify(codecs.decode(result, 'hex'))

def point_double():
    print("do this")

def point_add():
    print("do that")

## Alice's Details and Message
In the examples of this document, Alice is sending a message to Bob. Her private key, $P_{A}^{DH}$, is defined as the variable `SA` in the code below.


In [58]:
SA = 54221739386215982513329628131540269007441981031353650007031898434875003141355

PA = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, \
                               26186473731199907689170338364739255189550659682973035734804284247663992809959, \
                               71894484974716313150658193896579683660742267101889515604922967520820712029133, 10)

CompPA = '0339e504d6492b082da96e11e8f039796b06cd4855c101e2492a6f10f3e056a9e7'

message = 'this is my test message'

# Bob's Details
Bob is the reciever.

In [59]:
SB = 19604546966363504345948514434989414564433960545638122907271979546904571708973

PB = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, \
                               60650801378751831856369186448697049492352593086860468010779747572550299104457, \
                               34835621696986631909597048749467048062767037690835130200819150414373328549789, 10)

CompPB = '03861723172e8f50bf4e3b2b70582619a7260b79f040bcb2460ffd1d9053f534c9'

## Establishing Shared Secret
**Input: The private key** $S_{A}^{DH}$ **and public key $P_{B}^{DH}$.**<br>
**Output: Shared Secret Data.**<br><br>
**Method:**<br>
The public key of Bob is multiplied by the private key of Alice to get a new point on the elliptic curve $S_{AB}=S_{A}^{DH}⋅P_{B}^{DH}$. This point is a secret that is only known to Alice and Bob, and should not be shared. From this the following shared secret data is obtained:
<h5 align="center"> $S = $ compressed format of $S_{AB}$ (33-bytes)</h5>



In [60]:
S = '031cc3959f09765b99d8323496bd1b27126c0dcc6ca369ec66b6cdef324301614a'

## Establishing symmetric keys
**Input: Shared Secret Data _S_.**<br>
**Output: IV, AES Key, HMAC Key.**<br><br>
**Method:**<br>
The secret data S is hashed by SHA512. It is then split into 128-bit blocks. These are used for the calculate
<h5 align="center"> IV = first 128-bits of SHA512(S),</h5>
<h5 align="center"> AES Key = second 128-bits of SHA512(S),</h5>
<h5 align="center"> HMAC Key = final 256-bits of SHA512(S)</h5>

<br>**Additional Remarks:**<br>
- Take note that the SHA512 function takes a bytes-like object only as its parameter. This may be a mutable 'bytearray' or immutable 'bytes' object.

In [68]:
data = bytearray.fromhex(S)

IV = hashlib.sha512(data).digest()[0:16]
KAES = hashlib.sha512(data).digest()[16:32]
KHMAC = hashlib.sha512(data).digest()[32:64]

#print("IV as bytes:    " + str(IV))
#print("KAES as bytes:  " + str(KAES))
#print("KHMAC as bytes: " + str(KHMAC))

print("IV as hex:      " + str(binascii.hexlify(IV)))
print("KAES as hex:    " + str(binascii.hexlify(KAES)))
print("KHMAC as hex:   " + str(binascii.hexlify(KHMAC)))

IV as hex:      b'4a76311614a90d81a7c4f8d038c91f5f'
KAES as hex:    b'a4375fefdde407b0728ca4d35475cd67'
KHMAC as hex:   b'94d936f43d8c2e22cc81b52f8c28bfd5ac5e74c45bf3cabfbf145fc0f0d92337'


## Message Encryption
**Input: Plaintext, IV, AES Key.**<br>
**Output: Ciphertext.**<br><br>
**Method:**<br>
The plaintext is encrypted using AES 128-bit CBC mode algorithm with the AES Key and IV as defined above. A rough outline of the method is as follows: 

1. The plaintext is split into 128-bit blocks, the last block of which may need to be padded. PKCS7 padding is used.
2. The first block is then XOR’d with the IV. This ensures that if the same plaintext is encrypted twice with the same AES Key then then a different ciphertext will be produced (assuming the IV is not the same).
3. The result is fed into the AES 128 algorithm.

<br>**Additional Remarks:**<br>
- Take note that the `aes.encrypt` function requires bytes only, not bytearray.


In [75]:
aes = AES.new(KAES, AES.MODE_CBC, IV) 
encoder = PKCS7Encoder() 
messagebuf = encoder.encode(message) 
messagebuf = binascii.hexlify(messagebuf.encode())
messagebuf = messagebuf.decode() 
print("messagebuf = " + messagebuf)
messagebuf = bytes.fromhex(messagebuf)

encryptedBytes = aes.encrypt(messagebuf)

ciphertext = binascii.hexlify(encryptedBytes)
print("\nciphertext: " + str(ciphertext.decode()))

messagebuf = 74686973206973206d792074657374206d657373616765090909090909090909

ciphertext: 75c73ade8bded2a6ad7868eb28faa617e888c9013a15257513e7108e1f4995db


## Creating Hmac
**Input: Ciphertext, HMAC Key.**<br>
**Output: HMAC.**<br><br>
**Method:**<br>
Let us denote the HMAC Key as $K_{HMAC}$ and the hash function SHA-256 as $H$.<br>
Define *opad* = 0x5c⋅16 and *ipad* = 0x36⋅16. Calculate
<h5 align="center">$HMAC' = H((K_{HMAC}⨁*opad*) ∥ (K_{HMAC}⨁*ipad*) ∥ plaintext)).$</h5>
<br>The HMAC is defined to be the first 128-bits of this<br>
<h5 align="center">HMAC = HMAC' restricted to first 128-bits.</h5>
<br>The data is concatentated in the following order: Protocol Flag, Public Key, Ciphertext, HMAC. See Entire package.

In [76]:
BIE1 = '42494531'
pubKey = CompPA
encbuf = BIE1 + pubKey + ciphertext.decode()

encbuf = bytes.fromhex(encbuf)
h = hmac.new(KHMAC, encbuf, hashlib.sha256).hexdigest()

encbuf = binascii.hexlify(encbuf)
encbuf = encbuf.decode()
encbuf = encbuf + h

print("\nEntire package as hex:\n" + encbuf)
x = codecs.decode(encbuf, "hex")
print("\nEntire package as base-64:\n" + str(base64.b64encode(x).decode()))


Entire package as hex:
424945310339e504d6492b082da96e11e8f039796b06cd4855c101e2492a6f10f3e056a9e775c73ade8bded2a6ad7868eb28faa617e888c9013a15257513e7108e1f4995dbbd9247832033c40e92a2e8ddbaf257bfe03dae2de9a3dbde8a5ae673ca7a770a

Entire package as base-64:
QklFMQM55QTWSSsILaluEejwOXlrBs1IVcEB4kkqbxDz4Fap53XHOt6L3tKmrXho6yj6phfoiMkBOhUldRPnEI4fSZXbvZJHgyAzxA6SoujduvJXv+A9ri3po9veilrmc8p6dwo=


## Acknowledgments
nChain Research and Engineering team staff, including (in no particular order): Liam, Allison, Owen, Alex, Jad.

## Known Errors
Problem with PKCS7 padding. In this implementation, PKCS7 does not pad the plaintext when the size of the plaintext is exactly a multiple of 16-bytes. The JavaScript version of this code (that utilizes electrum-ecies lib) does pad in this instance, thus a discrepancy is caused.